In [1]:
print("Hello Jupyter")

Hello Jupyter


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
import requests
import json

url = "https://api.openai.com/v1/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + openai_api_key
}
data = {
    "model": "gpt-3.5-turbo",
    "prompt": "hello!",
    "temperature": 0
}

response = requests.post(url=url, headers=headers, json=data)
print(json.dumps(response.json(), indent=2))

{
  "error": {
    "message": "This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?",
    "type": "invalid_request_error",
    "param": "model",
    "code": null
  }
}


In [5]:
"""
import requests
import json

url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + openai_api_key
}
data = {
    "model": "gpt-3.5-turbo",
    "messages": [
        {"role" : "user", "content": "hello!"}
    ],
    "temperature": 0
}

response = requests.post(url=url, headers=headers, json=data)
print(json.dumps(response.json(), indent=2))
"""

'\nimport requests\nimport json\n\nurl = "https://api.openai.com/v1/chat/completions"\nheaders = {\n    "Content-Type": "application/json",\n    "Authorization": "Bearer " + openai_api_key\n}\ndata = {\n    "model": "gpt-3.5-turbo",\n    "messages": [\n        {"role" : "user", "content": "hello!"}\n    ],\n    "temperature": 0\n}\n\nresponse = requests.post(url=url, headers=headers, json=data)\nprint(json.dumps(response.json(), indent=2))\n'

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
result = llm.invoke("自己紹介してください。")
print(result)

content='こんにちは！私はAIアシスタントのChatGPTです。さまざまな質問に答えたり、情報を提供したり、会話を楽しんだりするために設計されています。趣味や興味、学びたいことについてお話しすることができますので、何でも気軽に聞いてください！あなたのサポートができることを楽しみにしています。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 11, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None} id='run-de847e43-b8d5-4a05-a54f-31776d201980-0' usage_metadata={'input_tokens': 11, 'output_tokens': 88, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [26]:
from langchain.prompts import PromptTemplate

template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""

prompt = PromptTemplate(
    input_variables=["command"],
    template=template
)

result = prompt.format(command="ls")
print(result)


次の子マントの概要を説明してください。

コマンド: ls



In [30]:
import langchain
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""

prompt = PromptTemplate(
    input_variables=["command"],
    template=template
)

chain = LLMChain(llm=chat, prompt=prompt)
result = chain.run("ls")
print(result)



> Entering new LLMChain chain...
Prompt after formatting:

次のコマンドの概要を説明してください。

コマンド: ls


> Finished chain.
`ls` コマンドは、Unix系のオペレーティングシステム（LinuxやmacOSなど）で使用されるコマンドで、指定したディレクトリ内のファイルやサブディレクトリの一覧を表示するためのものです。デフォルトでは、現在の作業ディレクトリの内容を表示します。

主な機能やオプションには以下のようなものがあります：

- `-l` オプション: 詳細情報を表示します（ファイルのパーミッション、所有者、サイズ、最終更新日時など）。
- `-a` オプション: 隠しファイル（ドットで始まるファイル）も含めてすべてのファイルを表示します。
- `-h` オプション: サイズを人間に読みやすい形式（KB, MBなど）で表示します。
- `-R` オプション: サブディレクトリも再帰的に表示します。

このように、`ls` コマンドはファイルシステムの内容を確認するための基本的かつ非常に便利なツールです。


In [31]:
cot_template = """
以下の質問に回答してください。

### 質問 ###
{question}
### 質問終了 ###

ステップバイステップで考えましょう。
"""

cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template
)
cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

summarize_template = """
入力を結論だけ一言に要約してください。

### 入力 ###
{input}
### 入力終了 ###
"""

summarize_prompt = PromptTemplate(
    input_variables=["inupt"],
    template=summarize_template
)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

In [32]:
from langchain.chains import SimpleSequentialChain

cot_summarize_chain = SimpleSequentialChain(chains=[cot_chain, summarize_chain])

result = cot_summarize_chain("私は市場に行って１０個のリンゴを買いました。隣人に２つ、修理工に２つ渡しました。それから５つのリンゴを買って１つ食べました。残りは何個ですか？")
print(result["output"])

/tmp/ipykernel_975/3858086507.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = cot_summarize_chain("私は市場に行って１０個のリンゴを買いました。隣人に２つ、修理工に２つ渡しました。それから５つのリンゴを買って１つ食べました。残りは何個ですか？")




> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

以下の質問に回答してください。

### 質問 ###
私は市場に行って１０個のリンゴを買いました。隣人に２つ、修理工に２つ渡しました。それから５つのリンゴを買って１つ食べました。残りは何個ですか？
### 質問終了 ###

ステップバイステップで考えましょう。


> Finished chain.
では、ステップバイステップで考えてみましょう。

1. **最初に買ったリンゴの数**: 10個
2. **隣人に渡したリンゴの数**: 2個
3. **修理工に渡したリンゴの数**: 2個

   ここで、渡したリンゴの合計は 2 + 2 = 4 個です。

4. **残りのリンゴの数**: 
   最初の10個から渡した4個を引きます。
   \[
   10 - 4 = 6 \text{個}
   \]

5. **その後、さらにリンゴを買った数**: 5個
6. **新しい合計のリンゴの数**: 
   残りの6個に新たに買った5個を加えます。
   \[
   6 + 5 = 11 \text{個}
   \]

7. **食べたリンゴの数**: 1個
8. **最終的な残りのリンゴの数**: 
   11個から食べた1個を引きます。
   \[
   11 - 1 = 10 \text{個}
   \]

したがって、最終的に残っているリンゴの数は **10個** です。


> Entering new LLMChain chain...
Prompt after formatting:

入力を結論だけ一言に要約してください。

### 入力 ###
では、ステップバイステップで考えてみましょう。

1. **最初に買ったリンゴの数**: 10個
2. **隣人に渡したリンゴの数**: 2個
3. **修理工に渡したリンゴの数**: 2個

   ここで、渡したリンゴの合計は 2 + 2 = 4 個です。

4. **残りのリンゴの数**: 
   最初の10個から渡した4個を引きます。
   \[
   10 - 4 = 6 \tex

In [35]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator
from typing import List

langchain.verbose = True
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

class Recipe(BaseModel):
    ingredients: List[str] = Field(description="ingredients of the dish")
    steps: List[str] = Field(description="steps to make the dish")

template = """料理のレシピを教えてください。
{format_instructions}

料理名: {dish}
"""

parser = PydanticOutputParser(pydantic_object=Recipe)

prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = LLMChain(llm=chat, prompt=prompt)
output = chain.run(dish="カレー")
print(output)



> Entering new LLMChain chain...
Prompt after formatting:
料理のレシピを教えてください。
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```

料理名: カレー


> Finished chain.
```json
{
  "ingredients": [
    "鶏肉 500g",
    "玉ねぎ 2個",
    "にんじん 1本",
    "じゃがいも 2個",
    "カレールー 1箱",
    "水 800ml",
    "サラダ油 大さじ2",
  

In [37]:
recipe = parser.parse(output)
print(recipe)
print(type(recipe))

ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、全体を混ぜる。', '水を加え、沸騰したらアクを取り、弱火で20分煮る。', 'カレールーを加え、よく溶かしてさらに10分煮る。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。']
<class '__main__.Recipe'>
